In [1]:
import os
import shutil
from pathlib import Path
from HandsFreeDocking.Wrapper_Docking import PipelineDocking
from HandsFreeDocking.tools.Ligand_Preparation import LigandPreparator

import datamol as dm
from rdkit import Chem

In [ ]:
protein_pdb = Path("./examples/LAG3_Moloc_2.pdb")
ligands_sdf = Path("./examples/Ligands_Complex.sdf")

cystal_sdf = Path("./examples/Fake_Crystal.sdf")

docking_dir = Path("./examples/TMP_Docking")

In [ ]:
if docking_dir.exists():
    shutil.rmtree(docking_dir, ignore_errors=True)

In [ ]:
docking = PipelineDocking(
    workdir=docking_dir,
    docking_software=["plants", "gnina", "rxdock"],
    settings=(4, 8),
    protein_pdb=protein_pdb,
    ligands_input=ligands_sdf,
    crystal_sdf=cystal_sdf,
    protonation_method="cdp",
    tautomer_score_threshold=None
)

In [ ]:
results = docking.run()
FULL_DF = docking.concat_df()

## Ligand Preparation

In [ ]:
smiles_to_test = [
    "C1C(C[NH3+])=CC2=C(C(N[C@@H](C3C=CC=C(C(=O)[O-])C=3)C)=NS2(=O)=O)C=1",
    "C1C(C[NH3+])=CC2=C(C(NC(C3C=CC=C(C(=O)[O-])C=3)C)=NS2(=O)=O)C=1",
    "N1C=C(C2CC2)C(=O)NC1=O",
    "C1C=CN2C(NC(C)C)=C(CC)N=C2C=1",
    "C1C=CN2C(NC(C)C)=CN=C2C=1",
    "C1=CC(=CC=C1C2=CC(=O)C3=C(C=C(C=C3O2)O)O)O"
]

names_of_smiles = [f"Lig_Complex_{i+1}" for i in range(len(smiles_to_test))]

assert len(smiles_to_test) == len(names_of_smiles), "The number of SMILES and names must match."

In [ ]:
preparator = LigandPreparator(
    protonation_method="cdp",
    enumerate_stereo=True,
    tautomer_score_threshold=None,
    generate_3d=True
)

# Prepare molecules from SDF
prepared_mols = preparator.prepare_from_smiles(smiles_to_test, names_of_smiles)
print(len(prepared_mols), "molecules prepared from SMILES.")

## MOL2 -> SDF Conv

In [2]:
import os
import shutil
from pathlib import Path
from HandsFreeDocking.Wrapper_Docking import PipelineDocking
from HandsFreeDocking.tools.Ligand_Preparation import LigandPreparator

In [3]:
from HandsFreeDocking.tools.Fix_Mol2 import hard_fix_mol2
from rdkit import Chem
from rdkit.Chem import AllChem
import datamol as dm
from typing import List, Tuple, Optional

In [4]:
mol2_example = Path("/home/hitesit/Python_Packages/Docking_Pipelines/HandsFreeDocking/examples/TMP_Docking/Gnina/output/Lig_Complex_1_Iso0_Gnina.sdf")
assert mol2_example.exists(), "The example mol2 file does not exist."

smiles_example = "C1=CC(=CC=C1C2=CC(=O)C3=C(C=C(C=C3O2)O)O)O"

In [ ]:
fixed_mols = hard_fix_mol2(mol2_example, smiles_example)

In [ ]:
X = dm.read_mol2file(str(mol2_example), sanitize=False)[1]

In [ ]:
AllChem.Compute2DCoords(X)

In [6]:
sdf_example = Path("examples/TMP_Docking/Gnina/output/Lig_Complex_1_Iso0_Gnina.sdf")